# EECS 498-007/598-005 Assignment 1-1: KNN SIFT

Before we start, please put your name and UMID in following format


**Your Answer:**   
Diwas Pandey, #011805103

#KNN with SIFT feature

The provided code loads the CIFAR-10 dataset, applies SIFT feature extraction to images, constructs a codebook using MiniBatchKMeans clustering, computes Bag of Visual Words (BoVW) representations, trains a K-Nearest Neighbors (K-NN) classifier, and evaluates scene recognition accuracy. It starts by preprocessing and loading the CIFAR-10 dataset. Then, it extracts SIFT features from the images, creates a codebook for clustering these features, and computes BoVW representations for both training and testing data. A K-NN classifier is trained on the training data, and its accuracy is evaluated on the test data, with results including accuracy and a classification report. Hyperparameters like the number of clusters, batch size, and the number of neighbors can be adjusted to optimize performance.

In [7]:
import numpy as np
import cv2
import torch
import torchvision
import torchvision.transforms as transforms
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score, classification_report
from sklearn.cluster import MiniBatchKMeans

def load_cifar10():
    # Step 1: Load CIFAR-10 dataset and preprocess it
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    # Download CIFAR-10 dataset
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

    # Extract labels from the dataset
    train_labels = np.array(trainset.targets)
    test_labels = np.array(testset.targets)

    return train_labels, test_labels, trainset, testset

def extract_sift_features(images):
    # Step 2: Extract SIFT features from CIFAR-10 images using OpenCV's SIFT
    sift = cv2.SIFT_create()
    sift_features = []

    for image in images:
        gray_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)
        kp, descriptors = sift.detectAndCompute(gray_image, None)
        if descriptors is not None:
            sift_features.append(descriptors)

    return sift_features

def create_codebook(features, num_clusters, batch_size):
    # Step 3: Create a codebook using MiniBatchKMeans clustering
    kmeans = MiniBatchKMeans(n_clusters=num_clusters, batch_size=batch_size, random_state=0)
    kmeans.fit(features)
    return kmeans

def compute_bovw_representation(features, codebook):
    # Step 4: Compute Bag of Visual Words (BoVW) representations for train and test data
    num_clusters = codebook.n_clusters
    bovw_representation = []

    for image_features in features:
        if len(image_features) > 0:
            # Assign each feature to a cluster
            cluster_assignments = codebook.predict(image_features)

            # Create a histogram of cluster frequencies
            histogram = np.bincount(cluster_assignments, minlength=num_clusters)

            # Normalize the histogram
            histogram = histogram / np.sum(histogram)

            bovw_representation.append(histogram)
        else:
            # Handle cases where no features were detected
            bovw_representation.append(np.zeros(num_clusters))

    return bovw_representation

def train_knn_classifier(train_bovw_features, train_labels):
    # Step 5: Create and train a Nearest Neighbors classifier (BallTree)
    knn_classifier = NearestNeighbors(n_neighbors=5, algorithm='ball_tree', n_jobs=-1)

    # Fit the classifier on the training data
    knn_classifier.fit(train_bovw_features)

    return knn_classifier

def evaluate_classifier(classifier, test_bovw_features, train_labels):
    # Step 6: Find nearest neighbors for test data
    distances, indices = classifier.kneighbors(test_bovw_features)

    # Step 7: Evaluate the classifier
    # Calculate the most common class label among neighbors
    predictions = []

    for neighbors in indices:
        if len(neighbors) > 0:
            neighbor_labels = train_labels[neighbors]
            most_common_label = np.bincount(neighbor_labels).argmax()
            predictions.append(most_common_label)
        else:
            # Handle cases where no neighbors were found
            predictions.append(-1)  # You can choose an appropriate value

    # Filter out test samples without predictions (-1)
    filtered_test_labels = []
    filtered_predictions = []

    for i, prediction in enumerate(predictions):
        if prediction != -1:
            filtered_test_labels.append(test_labels[i])
            filtered_predictions.append(prediction)

    # Calculate accuracy
    accuracy = accuracy_score(filtered_test_labels, filtered_predictions)
    report = classification_report(filtered_test_labels, filtered_predictions)

    return accuracy, report

def main():
    train_labels, test_labels, trainset, testset = load_cifar10()
    train_sift_features = extract_sift_features(trainset.data)
    test_sift_features = extract_sift_features(testset.data)

    num_clusters = 100  # You can adjust the number of clusters
    batch_size = 1000   # You can adjust the batch size

    concatenated_train_sift_features = np.vstack(train_sift_features)
    codebook = create_codebook(concatenated_train_sift_features, num_clusters, batch_size)

    train_bovw_features = compute_bovw_representation(train_sift_features, codebook)
    test_bovw_features = compute_bovw_representation(test_sift_features, codebook)

    knn_classifier = train_knn_classifier(train_bovw_features, train_labels)

    accuracy, report = evaluate_classifier(knn_classifier, test_bovw_features, train_labels)

    print("Accuracy:", accuracy)
    print("Classification Report:\n", report)

if __name__ == "__main__":
    main()


Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Accuracy: 0.10374524334067695
Classification Report:
               precision    recall  f1-score   support

           0       0.11      0.26      0.15       998
           1       0.10      0.17      0.13       999
           2       0.11      0.12      0.12       999
           3       0.08      0.07      0.08       997
           4       0.08      0.06      0.07      1000
           5       0.11      0.08      0.09       998
           6       0.14      0.08      0.10      1000
           7       0.11      0.07      0.09       998
           8       0.09      0.05      0.07       997
           9       0.11      0.07      0.08      1000

    accuracy                           0.10      9986
   macro avg       0.10      0.10      0.10      9986
weighted avg       0.10      0.10      0.10      9986



The accuracy achieved with the provided code is relatively low, approximately 10.37%. The classification report further demonstrates that the model's precision, recall, and F1-score for each class are quite low, indicating that the model is struggling to distinguish between different categories in the CIFAR-10 dataset.